## Central Türkiye Earthquakes (2023)

The InSAR.dev ecosystem, the PyGMTSAR InSAR library, the Geomed3D geophysical inversion library, and N-Cube 3D/4D GIS data visualization (among others) are open-source projects I develop in my free time.

I hold a Master’s degree in STEM, specializing in radio physics. In 2004, I received first prize in the All-Russian Physics Competition for significant results in forward and inverse modeling for nonlinear optics and holography. These skills are also applicable to modeling gravity, magnetic, and thermal fields, as well as satellite interferometry processing.

With 20 years of experience as a data scientist and software developer, I have contributed to scientific and industrial development through government contracts, university projects, and work with companies including LG Corp and Google Inc.

You can support my work on Patreon, where I share updates on my projects, publications, use cases, examples, and other useful information. For research and development services and support, please visit my profile on the freelance platform Upwork.

You can support my work on [Patreon](https://www.patreon.com/pechnikov), where I share updates on my projects, publications, use cases, examples, and other useful information. For research and development services and support, please visit my profile on the freelance platform [Upwork](https://www.upwork.com) or reach out to me directly.

### Resources
- Google Colab Pro notebooks and articles on [Patreon](https://www.patreon.com/pechnikov),
- Google Colab notebooks on [GitHub](https://github.com),
- Docker Images on [DockerHub](https://hub.docker.com),
- Geological Models on [YouTube](https://www.youtube.com),
- VR/AR Geological Models on [GitHub](https://github.com),
- Live updates and announcements on [LinkedIn](https://www.linkedin.com/in/alexey-pechnikov/).

© Alexey Pechnikov, 2025

$\large\color{blue}{\text{Hint: Use menu Cell} \to \text{Run All or Runtime} \to \text{Complete All or Runtime} \to \text{Run All}}$
$\large\color{blue}{\text{(depending of your localization settings) to execute the entire notebook}}$

# Stage 1. InSAR.dev-PyGMTSAR: A Python package for InSAR pre-processing with PyGMTSAR

Convert Sentinel-1 SLC data using GMTSAR binaries into a geocoded, cloud-ready Zarr dataset on Google Colab or in a Docker container. The output can be hosted on GitHub or any file storage as a set of files or a single ZIP archive.

For Stage 2 InSAR analysis, only a pure-Python package is required—no binary installation needed—so processing can run on any Windows, macOS, or Linux host.

## Google Colab Installation

Install InSAR.dev Python libraries and required GMTSAR binaries

In [ ]:
import sys
if 'google.colab' in sys.modules:
    # install the exact commit (no pip cache)
    !{sys.executable} -m pip install --no-cache-dir \
      "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev_toolkit"
    !{sys.executable} -m pip install --no-cache-dir \
      "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev_pygmtsar"
    !{sys.executable} -m pip install --no-cache-dir \
      "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev"

In [ ]:
import platform, sys, os
if 'google.colab' in sys.modules:
    # script URL: https://github.com/AlexeyPechnikov/InSARdev/blob/main/insardev_pygmtsar/insardev_pygmtsar/data/google_colab.sh
    import importlib.resources as resources
    with resources.as_file(resources.files('insardev_pygmtsar.data') / 'google_colab.sh') as google_colab_script_filename:
        !sh {google_colab_script_filename}
    from google.colab import output
    output.enable_custom_widget_manager()

# specify GMTSAR installation path
PATH = os.environ['PATH']
if PATH.find('GMTSAR') == -1:
    PATH = os.environ['PATH'] + ':/usr/local/GMTSAR/bin/'
    %env PATH {PATH}

## Load and Setup Python Modules

In [ ]:
import numpy as np
import geopandas as gpd
import shapely
import json
import matplotlib.pyplot as plt
%matplotlib inline
# setup dark theme
from insardev.UI import UI
UI('dark')

In [ ]:
# print versions
from insardev_pygmtsar import __version__ as pygmtsar_version
from insardev_toolkit import __version__ as toolkit_version
print("insardev_pygmtsar version:", pygmtsar_version)
print("insardev_toolkit version:", toolkit_version)
# import modules to be used in the notebook
from insardev_pygmtsar import S1
from insardev_toolkit import EOF, ASF, Tiles, XYZTiles

## Specify Sentinel-1 SLC Bursts and Area

### Descending Orbit Bursts

https://search.asf.alaska.edu/#/?polygon=LINESTRING(35.7%2036,37%2038.8,36.7%2035.8,38.7%2038.5,38%2035.5)&searchType=Geographic%20Search&searchList=S1A_IW_SLC__1SDV_20230129T033427_20230129T033455_046993_05A2FE_6FF2,S1A_IW_SLC__1SDV_20230129T033452_20230129T033519_046993_05A2FE_BE0B,S1A_IW_SLC__1SDV_20230210T033426_20230210T033454_047168_05A8CD_FAA6,S1A_IW_SLC__1SDV_20230210T033451_20230210T033518_047168_05A8CD_E5B0&resultsLoaded=true&granule=S1_043820_IW1_20230129T033512_VV_BE0B-BURST&zoom=6.843&center=36.353,34.829&start=2023-01-28T17:00:00Z&end=2023-02-10T16:59:59Z&flightDirs=Descending&dataset=SENTINEL-1%20BURSTS&path=21-21&beamModes=IW&polarizations=VV

In [ ]:
# Specify bursts to download
BURSTS = """
S1_043822_IW1_20230210T033516_VV_D767-BURST
S1_043821_IW3_20230210T033515_VV_E5B0-BURST
S1_043821_IW2_20230210T033514_VV_E5B0-BURST
S1_043821_IW1_20230210T033513_VV_E5B0-BURST
S1_043820_IW3_20230210T033513_VV_E5B0-BURST
S1_043820_IW2_20230210T033512_VV_E5B0-BURST
S1_043820_IW1_20230210T033511_VV_E5B0-BURST
S1_043819_IW3_20230210T033510_VV_E5B0-BURST
S1_043819_IW2_20230210T033509_VV_E5B0-BURST
S1_043819_IW1_20230210T033508_VV_E5B0-BURST
S1_043818_IW3_20230210T033507_VV_E5B0-BURST
S1_043818_IW2_20230210T033506_VV_E5B0-BURST
S1_043818_IW1_20230210T033505_VV_E5B0-BURST
S1_043817_IW3_20230210T033504_VV_E5B0-BURST
S1_043817_IW2_20230210T033503_VV_E5B0-BURST
S1_043817_IW1_20230210T033502_VV_E5B0-BURST
S1_043816_IW3_20230210T033502_VV_E5B0-BURST
S1_043816_IW2_20230210T033501_VV_E5B0-BURST
S1_043816_IW1_20230210T033500_VV_E5B0-BURST
S1_043815_IW3_20230210T033459_VV_E5B0-BURST
S1_043815_IW2_20230210T033458_VV_E5B0-BURST
S1_043815_IW1_20230210T033457_VV_E5B0-BURST
S1_043814_IW3_20230210T033456_VV_E5B0-BURST
S1_043814_IW2_20230210T033455_VV_E5B0-BURST
S1_043814_IW1_20230210T033454_VV_E5B0-BURST
S1_043813_IW3_20230210T033453_VV_E5B0-BURST
S1_043813_IW2_20230210T033452_VV_E5B0-BURST
S1_043813_IW1_20230210T033451_VV_E5B0-BURST
S1_043812_IW3_20230210T033451_VV_FAA6-BURST
S1_043812_IW2_20230210T033450_VV_FAA6-BURST
S1_043812_IW1_20230210T033449_VV_FAA6-BURST
S1_043811_IW3_20230210T033448_VV_FAA6-BURST
S1_043811_IW2_20230210T033447_VV_FAA6-BURST
S1_043811_IW1_20230210T033446_VV_FAA6-BURST
S1_043810_IW3_20230210T033445_VV_FAA6-BURST
S1_043810_IW2_20230210T033444_VV_FAA6-BURST
S1_043810_IW1_20230210T033443_VV_FAA6-BURST
S1_043809_IW3_20230210T033442_VV_FAA6-BURST
S1_043809_IW2_20230210T033441_VV_FAA6-BURST
S1_043809_IW1_20230210T033440_VV_FAA6-BURST
S1_043808_IW3_20230210T033439_VV_FAA6-BURST
S1_043808_IW2_20230210T033439_VV_FAA6-BURST
S1_043808_IW1_20230210T033438_VV_FAA6-BURST
S1_043807_IW3_20230210T033437_VV_FAA6-BURST
S1_043807_IW2_20230210T033436_VV_FAA6-BURST
S1_043807_IW1_20230210T033435_VV_FAA6-BURST
S1_043806_IW3_20230210T033434_VV_FAA6-BURST
S1_043806_IW2_20230210T033433_VV_FAA6-BURST
S1_043806_IW1_20230210T033432_VV_FAA6-BURST
S1_043805_IW3_20230210T033431_VV_FAA6-BURST
S1_043805_IW2_20230210T033430_VV_FAA6-BURST
S1_043805_IW1_20230210T033429_VV_FAA6-BURST
S1_043804_IW3_20230210T033428_VV_FAA6-BURST
S1_043804_IW2_20230210T033427_VV_FAA6-BURST
S1_043804_IW1_20230210T033427_VV_FAA6-BURST
S1_043803_IW3_20230210T033426_VV_FAA6-BURST
S1_043822_IW1_20230129T033517_VV_E089-BURST
S1_043821_IW3_20230129T033516_VV_BE0B-BURST
S1_043821_IW2_20230129T033515_VV_BE0B-BURST
S1_043821_IW1_20230129T033514_VV_BE0B-BURST
S1_043820_IW3_20230129T033513_VV_BE0B-BURST
S1_043820_IW2_20230129T033512_VV_BE0B-BURST
S1_043820_IW1_20230129T033512_VV_BE0B-BURST
S1_043819_IW3_20230129T033511_VV_BE0B-BURST
S1_043819_IW2_20230129T033510_VV_BE0B-BURST
S1_043819_IW1_20230129T033509_VV_BE0B-BURST
S1_043818_IW3_20230129T033508_VV_BE0B-BURST
S1_043818_IW2_20230129T033507_VV_BE0B-BURST
S1_043818_IW1_20230129T033506_VV_BE0B-BURST
S1_043817_IW3_20230129T033505_VV_BE0B-BURST
S1_043817_IW2_20230129T033504_VV_BE0B-BURST
S1_043817_IW1_20230129T033503_VV_BE0B-BURST
S1_043816_IW3_20230129T033502_VV_BE0B-BURST
S1_043816_IW2_20230129T033501_VV_BE0B-BURST
S1_043816_IW1_20230129T033501_VV_BE0B-BURST
S1_043815_IW3_20230129T033500_VV_BE0B-BURST
S1_043815_IW2_20230129T033459_VV_BE0B-BURST
S1_043815_IW1_20230129T033458_VV_BE0B-BURST
S1_043814_IW3_20230129T033457_VV_BE0B-BURST
S1_043814_IW2_20230129T033456_VV_BE0B-BURST
S1_043814_IW1_20230129T033455_VV_BE0B-BURST
S1_043813_IW3_20230129T033454_VV_BE0B-BURST
S1_043813_IW2_20230129T033453_VV_BE0B-BURST
S1_043813_IW1_20230129T033452_VV_BE0B-BURST
S1_043812_IW3_20230129T033451_VV_6FF2-BURST
S1_043812_IW2_20230129T033450_VV_6FF2-BURST
S1_043812_IW1_20230129T033449_VV_6FF2-BURST
S1_043811_IW3_20230129T033449_VV_6FF2-BURST
S1_043811_IW2_20230129T033448_VV_6FF2-BURST
S1_043811_IW1_20230129T033447_VV_6FF2-BURST
S1_043810_IW3_20230129T033446_VV_6FF2-BURST
S1_043810_IW2_20230129T033445_VV_6FF2-BURST
S1_043810_IW1_20230129T033444_VV_6FF2-BURST
S1_043809_IW3_20230129T033443_VV_6FF2-BURST
S1_043809_IW2_20230129T033442_VV_6FF2-BURST
S1_043809_IW1_20230129T033441_VV_6FF2-BURST
S1_043808_IW3_20230129T033440_VV_6FF2-BURST
S1_043808_IW2_20230129T033439_VV_6FF2-BURST
S1_043808_IW1_20230129T033438_VV_6FF2-BURST
S1_043807_IW3_20230129T033438_VV_6FF2-BURST
S1_043807_IW2_20230129T033437_VV_6FF2-BURST
S1_043807_IW1_20230129T033436_VV_6FF2-BURST
S1_043806_IW3_20230129T033435_VV_6FF2-BURST
S1_043806_IW2_20230129T033434_VV_6FF2-BURST
S1_043806_IW1_20230129T033433_VV_6FF2-BURST
S1_043805_IW3_20230129T033432_VV_6FF2-BURST
S1_043805_IW2_20230129T033431_VV_6FF2-BURST
S1_043805_IW1_20230129T033430_VV_6FF2-BURST
S1_043804_IW3_20230129T033429_VV_6FF2-BURST
S1_043804_IW2_20230129T033428_VV_6FF2-BURST
S1_043804_IW1_20230129T033427_VV_6FF2-BURST
S1_043803_IW3_20230129T033427_VV_6FF2-BURST
"""
BURSTS = list(filter(None, BURSTS.split('\n')))
print (f'Bursts defined: {len(BURSTS)}')

In [ ]:
EPICENTERS = [37.24, 38.11,37.08, 37.17]
POI = gpd.GeoDataFrame(geometry=[shapely.geometry.Point(coord) for coord in np.reshape(EPICENTERS, (2,-1))]).set_crs('EPSG:4326')
POI

## Specify Directories

In [ ]:
# directory where Sentinel-1 bursts and orbits will be downloaded
DATADIR = 'data'
# path to the downloaded DEM file
DEM = f'{DATADIR}/dem.nc'
ZARRDIR = 'zarr'

## Download and Unpack Datasets

### Enter Your ASF User and Password

If the data directory is empty or doesn't exist, you'll need to download Sentinel-1 scenes from the Alaska Satellite Facility (ASF) datastore. Use your Earthdata Login credentials. If you don't have an Earthdata Login, you can create one at https://urs.earthdata.nasa.gov//users/new

You can also use pre-existing SLC scenes stored on your Google Drive, or you can copy them using a direct public link from iCloud Drive.

The credentials below are available at the time the notebook is validated.

In [ ]:
# Set these variables to None and you will be prompted to enter your username and password below.
asf_username = 'GoogleColab2023'
asf_password = 'GoogleColab_2023'

In [ ]:
# Set these variables to None and you will be prompted to enter your username and password below.
asf = ASF(asf_username, asf_password)
print(asf.download(DATADIR, BURSTS))

In [ ]:
# read the notices printed below carefully
s1 = S1(DATADIR)
s1.to_dataframe().to_file(f"{DATADIR}/s1.geojson", driver="GeoJSON")
#df = gpd.read_file("s1.geojson")
s1.to_dataframe()

In [ ]:
# scan the data directory for Sentinel-1 SLC bursts and download related orbits
EOF().download(DATADIR, s1.to_dataframe())

In [ ]:
s1.to_dataframe()

In [ ]:
# download Copernicus Global DEM 1 arc-second
Tiles().download_dem(s1.to_dataframe(), provider='GLO', filename=DEM)[::4,::4].plot.imshow(cmap='cividis')
POI.plot(marker='*', color='red', markersize=50, ax=plt.gca());

## Preprocess Sentinel-1 SLC and Store as Geocoded Zarr

In [ ]:
# scan SLC bursts with DEM
s1 = S1(DATADIR, DEM=DEM)
s1.to_dataframe()

In [ ]:
# descending orbit bursts: preview
s1.plot(ref='2023-01-29')
POI.plot(marker='*', color='red', markersize=50, ax=plt.gca());

In [ ]:
# descending orbit bursts: load SLC bursts with DEM and transform to ZARR
# use lower resolution for Google Colab, default is (20, 5)
# the /tmp directory is extremely slow on Google Colab; use a RAM disk for temporary files
s1.transform(ZARRDIR, ref='2023-01-29', resolution=(100,25), epsg=32637,
             n_jobs=os.cpu_count() // 2 if 'google.colab' in sys.modules else None,
             tmpdir='/dev/shm' if 'google.colab' in sys.modules else None)

# Stage 2. InSAR.dev: A Pure-Python package for InSAR processing

For InSAR analysis, only a pure-Python package is required—no binary installation needed—so processing can run on any Windows, macOS, or Linux host.

## Load and Setup Python Modules

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# setup dark theme
from insardev.UI import UI
UI('dark')

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !{sys.executable} -m pip install --no-cache-dir \
        "git+https://github.com/AlexeyPechnikov/InSARdev.git#subdirectory=insardev"

In [ ]:
# print versions
from insardev import __version__ as insardev_version
from insardev_toolkit import __version__ as toolkit_version
print("insardev version:", insardev_version)
print("insardev_toolkit version:", toolkit_version)
# import modules to be used in the notebook
from insardev import Stack, BatchUnit
# data downloader
from insardev_toolkit.HTTP import unzip

In [ ]:
# common data science libraries
import xarray as xr
import numpy as np
import rioxarray as rio

In [ ]:
# 3D plotting modules
import pyvista as pv
# white background
#pv.set_plot_theme("document")
pv.set_plot_theme("dark")
from IPython.display import display, HTML
if 'google.colab' in sys.modules:
    import panel
    panel.extension(comms='ipywidgets')
    panel.extension('vtk')

## Specify Directories

In [ ]:
ZARRDIR = 'zarr'
WORKDIR = 'workdir'
LAND= f'{WORKDIR}/land.nc'

## Download and Unpack Datasets

We can process datasets hosted on GitHub, Zenodo, or other platforms in a single workflow.

In [ ]:
# example downloading command
#unzip("https://zenodo.org/records/15347694/files/Türkiye_Elevation-40x10-004.zip", ZARRDIR)

In [ ]:
# download land mask 1 arc-second
Tiles().download_landmask(s1.to_dataframe(), filename=LAND, product='1s')[::4,::4].plot.imshow(cmap='binary_r')
POI.plot(marker='*', color='red', markersize=50, ax=plt.gca());

## Run Local Dask Cluster

Launch a Dask cluster for local or distributed multicore computing. This makes it possible to process terabyte-scale Sentinel-1 SLC datasets even on an Apple MacBook Air with 16 GB of RAM.

In [ ]:
# simple Dask initialization
if 'client' in globals():
    client.shutdown()
from dask.distributed import Client
client = Client(silence_logs='CRITICAL')
client

## InSAR.dev Processing

In [ ]:
stack = Stack()
stack.load(ZARRDIR)
stack

In [ ]:
stack.plot(cmap='autumn', alpha=0.15)
POI.to_crs(stack.crs).plot(marker='*', color='red', markersize=50, ax=plt.gca())
# download the basemap adding the buffer to cover the area of interest after reprojecting
gmap = XYZTiles().download_googlesatellite(stack.to_dataframe().buffer(10_000), zoom=8, fill_value=0)
gmap.plot.imshow(ax=plt.gca(), zorder=-1, add_labels=False)
plt.gca().set_title(f'Sentinel-1 Burst Footprints');

## Build Interferogram

In [ ]:
# lazy calculation
intf, corr = stack.phasediff_multilook([0, 1], wavelength=400, goldstein=16)
intf

In [ ]:
# load land mask and convert to binary mask
# h5netcdf engine is required to read netCDF files on Google Colab
landmask = np.isfinite(xr.open_dataarray(LAND, engine='h5netcdf').rio.reproject(intf.crs))

In [ ]:
# mask, downsample to 100 meter and materialize interferogram and correlation
intf, corr = stack.compute(intf.mask(landmask).downsample(100), corr.mask(landmask).downsample(100))

In [ ]:
# align and unify bursts
intf = intf.align().dissolve().compute()
corr = corr.dissolve().compute()

In [ ]:
intf.plot()
POI.to_crs(stack.crs).plot(marker='*', color='red', markersize=50, ax=plt.gca());

In [ ]:
corr.plot()
POI.to_crs(stack.crs).plot(marker='*', color='red', markersize=50, ax=plt.gca());

In [ ]:
# load bursts elevation, downsample to 100 meter, align and unify bursts, materialize
elevation = stack.transform()[['ele']].downsample(100).align().dissolve().compute()

In [ ]:
elevation.plot(alpha=0.8)
POI.to_crs(stack.crs).plot(marker='*', color='red', markersize=50, ax=plt.gca());

## Unwrap Interferogram to Phase

In [ ]:
# unwrap as a single raster on auto-detected GPU or fallback to CPU
phase2d = stack.unwrap2d_dataset(intf.to_dataset(), corr.to_dataset(), device='auto')
phase2d

In [ ]:
# convert back to burst stack and materialize
phase = intf.from_dataset(phase2d).compute()

In [ ]:
phase.plot(quantile=[0.01, 0.99], alpha=0.8)
POI.to_crs(stack.crs).plot(marker='*', color='red', markersize=50, ax=plt.gca());

## Convert Phase to Displacement

In [ ]:
los = stack.displacement_los(phase)

In [ ]:
los.plot(alpha=0.8)
POI.to_crs(stack.crs).plot(marker='*', color='red', markersize=50, ax=plt.gca());

## 2D Interactive Map

In [ ]:
stack.to_vtk('intf', intf.downsample(400), elevation.downsample(400))
stack.to_vtk('los', los.downsample(400), elevation.downsample(400))

In [ ]:
# build interactive 3D plot
plotter = pv.Plotter(shape=(1, 2), notebook=True)

plotter.subplot(0, 0)
mesh = pv.read('intf/VV.vtk').scale([1, 1, 4], inplace=True)
plotter.add_mesh(mesh, scalars='20230129_20230210', cmap='turbo', ambient=0.1, show_scalar_bar=True, scalar_bar_args={'title': 'Interferogram [rad]'})
bounds = mesh.bounds
center = mesh.center
# move camera closer by reducing distance (zoom in)
distance = mesh.length / 1.4
plotter.camera.position = (center[0] + distance, center[1] + distance, center[2] + distance * 0.5)
plotter.camera.focal_point = center
plotter.camera.azimuth = 215
plotter.camera.elevation = 15

plotter.subplot(0, 1)
mesh = pv.read('los/VV.vtk').scale([1, 1, 4], inplace=True)
plotter.add_mesh(mesh, scalars='20230129_20230210', cmap='turbo', ambient=0.1, show_scalar_bar=True, scalar_bar_args={'title': 'LOS displacement [m]'})
bounds = mesh.bounds
center = mesh.center
# move camera closer by reducing distance (zoom in)
distance = mesh.length / 1.4
plotter.camera.position = (center[0] + distance, center[1] + distance, center[2] + distance * 0.5)
plotter.camera.focal_point = center
plotter.camera.azimuth = 215
plotter.camera.elevation = 15

#plotter.show_axes()
plot = plotter.show(screenshot='3D Elevation.png', jupyter_backend='panel' if 'google.colab' in sys.modules else 'client', return_viewer=True)
if 'google.colab' in sys.modules:
    plot = panel.panel(
        plotter.render_window, orientation_widget=plotter.renderer.axes_enabled,
        enable_keybindings=False, sizing_mode='stretch_width', min_height=600
    )
display(HTML('<h1 style="text-align:center; margin-bottom:0;">Sentinel-1 Interferogram and LOS Displacement on DEM, meters</h1>'))
plot

## Save the Results

Save the results in geospatial data formats like to NetCDF, GeoTIFF and others. The both formats (NetCDF and GeoTIFF) can be opened in QGIS and other GIS applications.

In [ ]:
los.to_dataset().VV[0].rio.to_raster('los.tif')
#los.to_dataset().VV[0].to_netcdf('los.nc')

## Export from Google Colab

In [ ]:
if 'google.colab' in sys.modules:
    from google.colab import files
    files.download('los.tif')